In [ ]:
#!pip install transformers
#!pip install sentencepiece
#!pip install torch

import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from sklearn.model_selection import train_test_split

data = pd.read_csv("drive/MyDrive/GPT-wiki-intro.csv")

# create a DataFrame from the list of dictionaries
df = data[['id', 'wiki_intro', 'generated_intro']].sample(n=1500, random_state=42)
df = df.rename(columns={'wiki_intro': 'source_text', 'generated_intro': 'target_text'})

#data.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 5.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Add prefix to source text
df['source_text'] = "summarize: " + df['source_text']

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.15)

# Instantiate the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Tokenize the source and target texts
train_encodings = tokenizer(list(train_df['source_text']), truncation=True, padding=True)
train_labels = tokenizer(list(train_df['target_text']), truncation=True, padding=True)

In [ ]:
# Fine-tune the model on the train set
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
for epoch in range(1):
    model.train()
    train_loss = 0
    for i in range(len(train_df)):
        input_ids = torch.tensor(train_encodings['input_ids'][i]).unsqueeze(0)
        attention_mask = torch.tensor(train_encodings['attention_mask'][i]).unsqueeze(0)
        labels = torch.tensor(train_labels['input_ids'][i]).unsqueeze(0)
        output = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = output.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Step {i} of {len(train_df)}")
    print(f"Epoch {epoch + 1} Train Loss: {train_loss / len(train_df)}")

Step 0 of 1275
Step 1 of 1275
Step 2 of 1275
Step 3 of 1275
Step 4 of 1275
Step 5 of 1275
Step 6 of 1275
Step 7 of 1275
Step 8 of 1275
Step 9 of 1275
Step 10 of 1275
Step 11 of 1275
Step 12 of 1275
Step 13 of 1275
Step 14 of 1275
Step 15 of 1275
Step 16 of 1275
Step 17 of 1275
Step 18 of 1275
Step 19 of 1275
Step 20 of 1275
Step 21 of 1275
Step 22 of 1275
Step 23 of 1275
Step 24 of 1275
Step 25 of 1275
Step 26 of 1275
Step 27 of 1275
Step 28 of 1275
Step 29 of 1275
Step 30 of 1275
Step 31 of 1275
Step 32 of 1275
Step 33 of 1275
Step 34 of 1275
Step 35 of 1275
Step 36 of 1275
Step 37 of 1275
Step 38 of 1275
Step 39 of 1275
Step 40 of 1275
Step 41 of 1275
Step 42 of 1275
Step 43 of 1275
Step 44 of 1275
Step 45 of 1275
Step 46 of 1275
Step 47 of 1275
Step 48 of 1275
Step 49 of 1275
Step 50 of 1275
Step 51 of 1275
Step 52 of 1275
Step 53 of 1275
Step 54 of 1275
Step 55 of 1275
Step 56 of 1275
Step 57 of 1275
Step 58 of 1275
Step 59 of 1275
Step 60 of 1275
Step 61 of 1275
Step 62 of 1275
St

In [ ]:
# Save the model to a file
torch.save(model.state_dict(), 'model.pth')

# Create a tar file and add the model file to it
with tarfile.open('model.tar', 'w') as tar:
    tar.add('model.pth')


In [ ]:
# Initialize an empty list to store the generated summaries
human_predictions = []

# Loop over each item in data['text']
for text in test_df['source_text']:
    # Encode the text using the tokenizer
    input_ids = tokenizer.encode(text, return_tensors='pt')
    
    # Generate a summary using the model
    summary_ids = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Add the generated summary to the list
    human_predictions.append(summary)

# Initialize an empty list to store the generated summaries
machine_predictions = []

# Loop over each item in data['text']
for text in test_df['target_text']:
    # Encode the text using the tokenizer
    input_ids = tokenizer.encode(text, return_tensors='pt')
    
    # Generate a summary using the model
    summary_ids = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Add the generated summary to the list
    machine_predictions.append(summary)

machine_predictions

t5_df = test_df.copy()
t5_df['human_reconstructed'] = human_predictions
t5_df['machine_reconstructed'] = machine_predictions
t5_df = t5_df.rename(columns={'source_text': 'human_original', 'target_text': 'machine_original'})
t5_df.to_csv('t5_df_final.csv', index=False)
